# Generate SP3 Input Parameters 
This file is used for generating SP3 input files using data from dyanamic analysis to perform FEMA P-58 loss assessment on SP3. 
Jointly lognormal distributions are assumed for all engineering demand parameters (EDPs).

In [1]:
# Import packages
import numpy as np
import math
import os
import pandas as pd
import sqlite3
import random

from scipy.stats import lognorm
from scipy.stats import binom
from scipy.stats import norm
from scipy.stats import truncnorm
from scipy.stats import uniform
from scipy.optimize import minimize
from scipy import interpolate

from Component import component # Predefined component class
import random
np.random.seed(1000)

def SampleEDP(aggregatedEDP, beta, NumRealizations):
    # This function is used for randomly sampling engineering demand parameters used for loss assessment
    ##############################################################################################################################################
    # Input:
    # aggregatedEDP      Aggregated EDPs at one intensity level, including SDR, PFA and RDR, attention: this function takes the original EDPs from the analsis
    # beta               Uncertainties, first row is model uncertainty, second row is ground motion uncertainty         
    # NumRealizations    Number of simulations at current intensity level
    # Output:
    # W                  Random sampled EDPs, including both collapse and non-collapse EDPs
    ##############################################################################################################################################
     
    EDPs = aggregatedEDP
    # ln scale of the input EDP
    lnEDPs = np.log(EDPs)

    # Number of rows
    num_rec = EDPs.shape[0]
    # Number of columns 
    num_var = EDPs.shape[1]

    lnEDPs_mean = np.array(np.mean(lnEDPs, axis = 0).T)
    lnEDPs_cov = np.cov(lnEDPs.T)

    lnEDPs_cov_rank = np.linalg.matrix_rank(lnEDPs_cov)

    # pay attention to the data format here, it has to be a column vector
    sigma = np.array(np.sqrt(np.diag(lnEDPs_cov))).reshape([num_var,1]) 
    sigmap2 = sigma * sigma

    R = lnEDPs_cov / (np.matmul(sigma,sigma.T))

    B = np.array(beta).reshape([1,2])

    # Incorporate with uncertainties
    sigmap2 = sigmap2 + B[:,0] * B[:,0]
    sigmap2 = sigmap2 + B[:,1] * B[:,1]
    sigma = np.sqrt(sigmap2)
    sigma2 = np.matmul(sigma , sigma.T) 
    lnEDPs_cov_inflated = R * sigma2


    D2_total,L_total = np.linalg.eig(lnEDPs_cov_inflated)

    idx = D2_total.argsort()  
    D2_total = D2_total[idx]
    L_total = L_total[:,idx]


    if lnEDPs_cov_rank >= num_var:
        L_use = L_total
    else:
        L_use = L_total[:, num_var - lnEDPs_covrank + 1 : num_var] # still have to check this part

    if lnEDPs_cov_rank >= num_var:
        D2_use = D2_total
    else: D2_use = D2_total[num_var - lnEDPs-cov_rank + 1: num_var] # still have to check this part

    D_use =np.diagflat(np.sqrt(D2_use))

    if lnEDPs_cov_rank >= num_var:
        U = np.random.randn(NumRealizations*num_var).reshape([num_var, NumRealizations])
    else: U = np.random.randn(NumRealizations, lnEDPs_cov_rank).reshape([num_var, NumRealizations])

    Lambda = -np.matmul(L_use , D_use)

    Z = np.matmul(Lambda , U) + np.repeat(lnEDPs_mean.reshape([num_var,1]), NumRealizations, axis=1)

    lnEDPs_sim_mean = np.mean(Z.T, axis = 0)
    lnEDPs_sim_cov = np.cov(Z)

    A = lnEDPs_sim_mean/lnEDPs_mean.T  #Table G-7, or Table G-16
    B = lnEDPs_sim_cov/lnEDPs_cov  #Table G-8, or Table G-17
    W = np.exp(Z)
    sampledEDP = pd.DataFrame(W.T,columns = aggregatedEDP.columns)
    return W

In [2]:
with open ('/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/BuildingName.txt') as f:
    BuildingName = f.read()
f.close()

BuildingName = BuildingName.split('\n')

In [3]:
# Loop over all input folders 
BaseDirectory = '/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/Updated Building Performance'
NumHazardLevel = 15

for p in range(1,97):
    os.chdir(BaseDirectory + '/case%i'%p)
    SDRTotal = pd.read_csv('SDR.csv', header = None)
    PFATotal = pd.read_csv('PFA.csv', header = None)
    RDRTotal = pd.read_csv('RDR.csv', header = None)
    NumStory = SDRTotal.shape[1]-3
    
    SDR_SP3 = pd.DataFrame(columns=SDRTotal.columns)
    SDR_SP3[0] = SDRTotal[0]
    SDR_SP3[1] = SDRTotal[1]
    SDR_SP3[2] = SDRTotal[2]
   
    PFA_SP3 = pd.DataFrame(columns=PFATotal.columns)
    PFA_SP3[0] = PFATotal[0]
    PFA_SP3[1] = PFATotal[1]
    PFA_SP3[2] = PFATotal[2]
    
    RDR_SP3 = pd.DataFrame(columns=RDRTotal.columns)
    RDR_SP3[0] = RDRTotal[0]
    RDR_SP3[1] = RDRTotal[1]
    RDR_SP3[2] = RDRTotal[2]
    
    tempSDR = np.empty([0,NumStory])
    tempPFA = np.empty([0,NumStory+1])
    tempRDR = np.empty([0])
    
    for i in range(1,16):
        SDRX = SDRTotal[(SDRTotal.loc[:,0] == i) & (SDRTotal.loc[:,1] == 1)]
        PFAX = PFATotal[(PFATotal.loc[:,0] == i) & (PFATotal.loc[:,1] == 1)]
        RDRX = RDRTotal[(RDRTotal.loc[:,0] == i) & (RDRTotal.loc[:,1] == 1)]

        SDRZ = SDRTotal[(SDRTotal.loc[:,0] == i) & (SDRTotal.loc[:,1] == 2)]
        PFAZ = PFATotal[(PFATotal.loc[:,0] == i) & (PFATotal.loc[:,1] == 2)]
        RDRZ = RDRTotal[(RDRTotal.loc[:,0] == i) & (RDRTotal.loc[:,1] == 2)]
        
        # Prepare the EDPs at current intensity level and sample EDPs
        SDRX_pure = SDRX.loc[:,3:SDRX.shape[1]]
        PFAX_pure = PFAX.loc[:,3:PFAX.shape[1]]
        RDRX_pure = RDRX.loc[:,3:RDRX.shape[1]]

        AggregatedXEDPs = pd.concat([SDRX_pure, PFAX_pure, RDRX_pure], axis=1)

        SDRZ_pure = SDRZ.loc[:,3:SDRZ.shape[1]]
        PFAZ_pure = PFAZ.loc[:,3:PFAZ.shape[1]]
        RDRZ_pure = RDRZ.loc[:,3:RDRZ.shape[1]]

        AggregatedZEDPs = pd.concat([SDRZ_pure, PFAZ_pure, RDRZ_pure], axis=1)

        temp = pd.concat([AggregatedXEDPs,AggregatedZEDPs], axis = 0, ignore_index=True)
        AggregatedEDP = pd.DataFrame(data = temp.values.reshape([AggregatedXEDPs.shape[0],AggregatedXEDPs.shape[1]*2]))
        
        SampledEDP = SampleEDP(AggregatedEDP, np.array([0,0]), 10000)
        sampledEDPXs = SampledEDP[0:int(SampledEDP.shape[0]/2)]
        Cur_SDRX = sampledEDPXs[0:NumStory]
        Cur_PFAX = sampledEDPXs[NumStory:NumStory+NumStory+1]
        Cur_RDRX = sampledEDPXs[-1]

        sampledEDPZs = SampledEDP[int(SampledEDP.shape[0]/2):SampledEDP.shape[0]]
        Cur_SDRZ = sampledEDPZs[0:NumStory]
        Cur_PFAZ = sampledEDPZs[NumStory:NumStory+NumStory+1]
        Cur_RDRZ = sampledEDPZs[-1]
        
        MaxSDR = np.maximum(np.max(Cur_SDRX, axis = 0),np.max(Cur_SDRZ, axis = 0))   
        
        
#         SDR_SP3[(SDR_SP3.loc[:,0] == i)].iloc[:,3:SDR_SP3.shape[1]] = 
#         np.concatenate((np.transpose(Cur_SDRX[:,MaxSDR<0.1][:,0:44]),np.transpose(Cur_SDRZ[:,MaxSDR<0.1][:,0:44])))
        
        tempSDR = np.concatenate((tempSDR,np.transpose(Cur_SDRX[:,MaxSDR<0.1][:,0:44])))
        tempSDR = np.concatenate((tempSDR,np.transpose(Cur_SDRZ[:,MaxSDR<0.1][:,0:44])))

        tempPFA = np.concatenate((tempPFA,np.transpose(Cur_PFAX[:,MaxSDR<0.1][:,0:44])))
        tempPFA = np.concatenate((tempPFA,np.transpose(Cur_PFAZ[:,MaxSDR<0.1][:,0:44])))
        
        tempRDR = np.concatenate((tempRDR,np.transpose(Cur_RDRX[MaxSDR<0.1][0:44])))
        tempRDR = np.concatenate((tempRDR,np.transpose(Cur_RDRZ[MaxSDR<0.1][0:44])))
        
        
    SDR_SP3.iloc[:,3:] = tempSDR
    PFA_SP3.iloc[:,3:] = tempPFA
    RDR_SP3.iloc[:,3:] = tempRDR

    os.chdir('/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/SP3 Input/%s'%BuildingName[p-1])
    
    SDR_SP3.to_csv('SDR.csv', header = None, index = False)
    PFA_SP3.to_csv('PFA.csv', header = None, index = False)
    RDR_SP3.to_csv('RDR.csv', header = None, index = False)
      
        

In [7]:
2%2+2

2

# Adjust Median Collapse Intensity per FEMA P-695

In [325]:
SSF = np.array([1.09571922, 1.01245261, 1.0930497 , 1.02471603, 1.10805072,
       1.02748051, 1.09393267, 1.03427207, 1.08386949, 1.02451625,
       1.08326639, 1.02455288, 1.1037251 , 1.02804048, 1.10156877,
       1.03669735, 1.08191171, 1.02766901, 1.08276446, 1.03136322,
       1.08701776, 1.02575623, 1.10124591, 1.02880187, 1.0833023 ,
       1.03166676, 1.0822175 , 1.03089174, 1.10448858, 1.024396  ,
       1.09811497, 1.02982435, 1.07952327, 1.02831036, 1.07723176,
       1.03009791, 1.10871187, 1.02744448, 1.09841607, 1.0336288 ,
       1.08562203, 1.02924885, 1.08320047, 1.03083144, 1.11670231,
       1.02818649, 1.11299566, 1.03992404, 1.08208485, 1.02949064,
       1.07724773, 1.02921092, 1.09171821, 1.02566203, 1.10887921,
       1.0285508 , 1.08420665, 1.02948647, 1.0829342 , 1.02941277,
       1.11828912, 1.0272267 , 1.11364835, 1.03257194, 1.08301218,
       1.0278338 , 1.07516192, 1.02886966, 1.11339058, 1.02704789,
       1.09757539, 1.03283428, 1.08393951, 1.02951265, 1.08316924,
       1.02944363, 1.10942837, 1.02876181, 1.11313005, 1.03862263,
       1.08240426, 1.0290608 , 1.07479885, 1.02921412, 1.08913195,
       1.02566268, 1.09014529, 1.02823359, 1.08331827, 1.028954  ,
       1.08276071, 1.02900985, 1.11722086, 1.02715933, 1.11235383,
       1.03294298])

In [331]:
BaseDirectory = '/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/Updated Building Performance'
NumHazardLevel = 15

for p in range(1,97):
    os.chdir(BaseDirectory + '/case%i'%p)
    CollapseMedian = pd.read_csv('CollapseFragility.csv',header = None)
    CollapseMedian.iloc[0,0] *= SSF[p-1]
    
    os.chdir('/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/SP3 Input/%s'%BuildingName[p-1])
    
    CollapseMedian.to_csv('CollapseFragility.csv',header = None)

In [332]:
CollapseMedian

,0
0,1.130442
1,0.229811


# Hazard Curve

In [355]:
HazardLevel = np.array([0.162958971,0.325917943,0.488876914,0.651835885,0.814794857,
                        0.977753828,1.140712799,1.303671771,1.466630742,1.629589713,
                        1.792548685,1.955507656,2.118466627,2.281425599,2.44438457])

Sa = np.array([0.0025,0.0045,0.0075,0.0113,0.0169,0.0253,0.038,0.057,0.0854,
                   0.128,0.192,0.288,0.432,0.649,0.973,
                   1.46,2.19,3.28,4.92,7.38])
Lambda = np.array([1.2858232,1.0615758,0.84598525,0.6784487,0.53093991,0.40578048,0.303132,0.22138945,0.15660468,0.10588062,
                   0.067680774,0.040523182,0.022339423,0.01099528,0.0047231629,0.0017479081,5.8443237E-4,1.9411083E-4,
                   6.5659132E-5,2.0956643E-5])


In [373]:
Hazard_rate = np.interp(HazardLevel, Sa, Lambda)
Return_period = 1/Hazard_rate
Prop_exceedance = 1-np.exp(-Hazard_rate)
Data = {'Sa':HazardLevel,
       'Prop of exceedence':Prop_exceedance,
       'Lambda':Hazard_rate,
       'return period': Return_period}
HazardInfo = pd.DataFrame(data = Data)

In [376]:
os.chdir('/Users/rover/Desktop/Cost_Benefit/20190701UpdateDesignForSeismicParameters/SP3 Input')
HazardInfo.to_csv('HazardCurve.csv')

In [377]:
Hazard_rate

array([0.08501457, 0.03573505, 0.01936606, 0.01094038, 0.00778576,
       0.00469412, 0.00369855, 0.00270297, 0.00173734, 0.00147762,
       0.00121789, 0.00095817, 0.00069844, 0.00055169, 0.00049334])